In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy
from pprint import pprint

<IPython.core.display.Javascript object>

In [2]:
csv_path = os.path.join("../data/csv/reviews_cleaned_reduced_500.csv")
TextFileReader = pd.read_csv(csv_path, chunksize=1000)  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)


In [3]:
# load beers
csv_path = os.path.join("../data/csv/beers.csv")
beers_df = pd.read_csv(csv_path)

In [4]:
beers_df = beers_df.rename(columns={'id': 'beer_id'})

In [5]:
# Lets combine the dataframe
merge_df = pd.merge(df,
                 beers_df[['beer_id', 'name', 'style', 'brewery_id']],
                 on='beer_id')

In [6]:
# merge_df.to_csv("../data/csv/beer_reviews_clean_500.csv",index=False)

In [7]:
# sample random trainset and testset method using Pearson similarity
# test set is made of 25% of the ratings. we are looking at similarities between items (user_based=false)
reader=Reader(rating_scale=(0,5))
data = Dataset.load_from_df(merge_df[['username', 'beer_id', 'score']], reader)

trainset, testset = train_test_split(data, test_size=.25)

sim_options = {'name': 'pearson_baseline',
               'user_based': False
               }

# We'll use KNN.
algo = KNNBasic(min_k = 10, sim_options=sim_options)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.fit(trainset).test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.4610
MAE:  0.3340


0.3340169743130181

In [8]:
# Five fold cross validation method to make sure our dataset is good
algo = KNNBasic(min_k = 10, sim_options=sim_options)

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4582  0.4550  0.4561  0.4584  0.4541  0.4564  0.0017  
MAE (testset)     0.3316  0.3297  0.3307  0.3314  0.3294  0.3305  0.0009  
Fit time          19.61   21.62   19.80   20.62   21.43   20.62   0.82    
Test time         37.79   38.47   37.39   42.18   40.70 

{'test_rmse': array([0.45818689, 0.45500242, 0.45613178, 0.45836846, 0.45414695]),
 'test_mae': array([0.33156492, 0.32968989, 0.33065911, 0.33136487, 0.32935831]),
 'fit_time': (19.608553886413574,
  21.61518359184265,
  19.803030252456665,
  20.624154329299927,
  21.42665457725525),
 'test_time': (37.7909152507782,
  38.47215414047241,
  37.394039154052734,
  42.17982602119446,
  40.6991753578186)}

In [58]:
# Lets serialize and save this prediction algorithm
# Dump algorithm and reload it
dumpfile = os.path.join('../data/dump/dump_knn_pearsonbaseline_500dump_file')
dump.dump(dumpfile, predictions, algo)

In [9]:
# Code below identifes the top 10 best and worst predictions based upon code from this 
# notebook:https://nbviewer.jupyter.org/github/NicolasHug/Surprise/blob/master/examples/notebooks/KNNBasic_analysis.ipynb
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
def get_inner_ids(riid):
    inner_ids = []
    for riid in riids:
        inner_ids.append(trainset.to_inner_iid(riid))
    return inner_ids
        
    
df_predict = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predict['Iu'] = df_predict.uid.apply(get_Iu)
df_predict['Ui'] = df_predict.iid.apply(get_Ui)
df_predict['err'] = abs(df_predict.est - df_predict.rui)
best_predictions = df_predict.sort_values(by='err')[:10]
worst_predictions = df_predict.sort_values(by='err')[-10:]

In [10]:
# Best Predictions:
print(best_predictions)

                 uid     iid  rui  est  \
58417      100200300     635  3.0  3.0   
12745       bmcduff2   48933  1.0  1.0   
34476       bmcduff2     754  1.0  1.0   
115476      bmcduff2     862  1.0  1.0   
169285      bmcduff2    1426  1.0  1.0   
303412      bcbs4040   88298  5.0  5.0   
47699      100200300   32575  3.0  3.0   
26787       thuglife   37265  5.0  5.0   
57452   DrunkyBuddha  145496  5.0  5.0   
233100  DrunkyBuddha    1708  5.0  5.0   

                                          details  Iu    Ui  err  
58417   {'actual_k': 29, 'was_impossible': False}  58   489  0.0  
12745   {'actual_k': 24, 'was_impossible': False}  39   407  0.0  
34476   {'actual_k': 26, 'was_impossible': False}  39  1702  0.0  
115476  {'actual_k': 34, 'was_impossible': False}  39   716  0.0  
169285  {'actual_k': 29, 'was_impossible': False}  39   529  0.0  
303412  {'actual_k': 10, 'was_impossible': False}  12   412  0.0  
47699   {'actual_k': 20, 'was_impossible': False}  58   386  0.0  
2

In [11]:
# Worst Predictions:
print(worst_predictions)

                      uid     iid  rui       est  \
256504       4thelvofbeer  100421  1.0  4.368577   
200692        cclark67789      65  1.0  4.394744   
123306           flipdog0    1545  1.0  4.417246   
258848       Jstonerock24    1320  1.0  4.420618   
286652         keags_beer   20762  1.0  4.429163   
213008  MarshallBirdhouse   86621  1.0  4.439916   
223263     robertjskelton   57747  1.0  4.463173   
168705        robertm2152   80197  1.0  4.495361   
169775         MaddDogg84   55019  1.0  4.619110   
73824                ucis   53728  1.0  4.975445   

                                          details   Iu    Ui       err  
256504  {'actual_k': 25, 'was_impossible': False}   35  1610  3.368577  
200692  {'actual_k': 21, 'was_impossible': False}   79  1453  3.394744  
123306  {'actual_k': 33, 'was_impossible': False}   76  1603  3.417246  
258848  {'actual_k': 21, 'was_impossible': False}   97  1369  3.420618  
286652  {'actual_k': 14, 'was_impossible': False}   26   395  

In [36]:
def get_beer_name (beer_raw_id):
    beer_name = beers_df.loc[beers_df.beer_id==beer_raw_id,'name'].values[0]
    return beer_name

def get_beer_style (beer_raw_id):
    beer_style = beers_df.loc[beers_df.beer_id==beer_raw_id,'style'].values[0]
    return beer_style

def get_beer_score_mean (beer_raw_id):
    score_mean = mean_score.loc[mean_score.beer_id==beer_raw_id,'score'].values[0]
    return score_mean

def get_beer_neighbors (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                  for inner_id in beer_neighbors)
    return(beer_neighbors)

def get_beer_recc_df (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                      for inner_id in beer_neighbors)
    beers_id_recc = []
    beer_name_recc =[]
    beer_style_recc = []
    beer_score_mean = []
    for beer in beer_neighbors:
        beers_id_recc.append(beer)
        beer_name_recc.append(get_beer_name(beer))
        beer_style_recc.append(get_beer_style(beer))
        beer_score_mean.append(get_beer_score_mean(beer))
    beer_reccomendations_df = pd.DataFrame(list(zip(beers_id_recc,beer_name_recc,beer_style_recc,beer_score_mean)),
                                       columns=['beer_id', 'name', 'style', 'score_mean'])
    return beer_reccomendations_df
    
def get_inner_ids(riids):
    inner_ids = []
    for riid in riids:
        inner_ids.append(trainset.to_inner_iid(riid))
    return inner_ids
        

In [13]:
#Create mean score dataframe and get riids (riids = raw beers ids)
mean_score = merge_df.groupby('beer_id', as_index=False)[['score']].mean()
riids = mean_score['beer_id']
riids = riids.to_list()
#Get the beer inner ids
inner_ids = get_inner_ids(riids)

In [ ]:
# Save the df_predict and df_ids for later use
df_predict.to_csv("../data/csv/df_predict_pearsonbaseline_500.csv",index=False)
df_ids = pd.DataFrame(list(zip(riids,inner_ids)), columns=['beer_id', 'inner_ids'])
df_ids.to_csv("../data/csv/df_ids_pearsonbaseline_500.csv",index=False)

In [54]:
# Create pickles of our dataframes to be used in the load program

filter_df = merge_df.drop(['username', 'score'], axis=1).drop_duplicates(subset =
                                                                        ['beer_id',
                                                                        'name',
                                                                        'style',
                                                                        'brewery_id'],
                                                                        keep="first")
iids_merged = pd.merge(df_ids, mean_score[['beer_id', 'score']], on='beer_id')

In [59]:
merged_iid_df = pd.merge(iids_merged,
                 filter_df[['beer_id', 'name', 'style', 'brewery_id']],
                 on='beer_id', )

In [57]:
merged_iid_df.to_pickle("../data/dump/beer.pkl")
df_predict.to_pickle('../data/dump/df_predict.pkl')

In [15]:
# Lets test some beers.  Enter a beer and use the predictions model to return 5 nearest neighbors
# K=10

beer_raw_id =  232 
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Corona Extra, American Adjunct Lager, score = 2.2471090047393374 are:


,beer_id,name,style,score_mean
0,233,Corona Light,American Light Lager,1.884826
1,1320,Bud Light,American Light Lager,1.790027
2,3734,Michelob Ultra,American Light Lager,1.795152
3,65,Budweiser,American Adjunct Lager,2.261485
4,1321,Modelo Especial,American Adjunct Lager,2.763934
5,2803,Sol,American Adjunct Lager,2.391426
6,1907,Dos Equis Special Lager,American Adjunct Lager,2.682793
7,41821,Bud Light Lime,American Light Lager,2.078862
8,689,Red Stripe Jamaican Lager,American Adjunct Lager,2.926540
9,246,Heineken Lager Beer,European Pale Lager,2.641871


In [16]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  412
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Old Rasputin, Russian Imperial Stout, score = 4.270477977977977 are:


,beer_id,name,style,score_mean
0,1655,Red Rocket Ale,American Amber / Red Ale,4.052642
1,57908,2XIPA,American Imperial IPA,3.986614
2,1160,Imperial Russian Stout,Russian Imperial Stout,4.314868
3,1658,Big Bear Black Stout,American Imperial Stout,4.117832
4,92,Arrogant Bastard Ale,American Strong Ale,4.132759
5,12013,Yeti Imperial Stout,American Imperial Stout,4.193738
6,283,Sierra Nevada Stout,American Stout,4.047086
7,21100,Oat (Imperial Oatmeal Stout),American Imperial Stout,4.080739
8,1446,Hibernation Ale,English Old Ale,4.012648
9,16115,Black Rye,Rye Beer,4.021536


In [17]:
   # Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors

beer_raw_id =  26049
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Wild Blue, Fruit and Field Beer, score = 2.1567888307155303 are:


,beer_id,name,style,score_mean
0,39908,Shock Top Belgian White,Belgian Witbier,2.856560
1,232,Corona Extra,American Adjunct Lager,2.247109
2,3734,Michelob Ultra,American Light Lager,1.795152
3,1932,Pumpkinhead Ale,Pumpkin Beer,2.926844
4,567,Rolling Rock Extra Pale,American Adjunct Lager,2.500788
5,670,Busch Beer,American Adjunct Lager,1.916544
6,5488,Wells Banana Bread Beer,Fruit and Field Beer,3.454176
7,100,Blue Moon Harvest Moon Pumpkin Ale,Pumpkin Beer,3.045953
8,76899,Bud Light Platinum,American Light Lager,2.112869
9,1907,Dos Equis Special Lager,American Adjunct Lager,2.682793


In [18]:
# Lets test some beers.  Enter a beer and use the predictions model to return 5 nearest neighbors

beer_raw_id =  6108
print(f'The 5 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 5 nearest neighbors of 60 Minute IPA, American IPA, score = 4.019886399495109 are:


,beer_id,name,style,score_mean
0,2093,90 Minute IPA,American Imperial IPA,4.220053
1,9086,120 Minute IPA,American Imperial IPA,4.060382
2,50764,Lucky U IPA,American IPA,3.434723
3,12068,Exponential Hoppiness,American Imperial IPA,4.395072
4,402,Skull Splitter,Scotch Ale / Wee Heavy,4.065929
5,26233,Fresh Hop Pale Ale,American Pale Ale (APA),4.063147
6,3951,ApriHop,American IPA,3.824306
7,1005,HopDevil India Pale Ale,American IPA,4.029686
8,115076,Milk Stout - Nitro,English Sweet / Milk Stout,4.067120
9,34203,Santa's Little Helper,Russian Imperial Stout,4.014268


In [19]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  332
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Miller Lite, American Light Lager, score = 2.06624165554072 are:


,beer_id,name,style,score_mean
0,837,Coors Light,American Light Lager,1.928670
1,580,Miller High Life,American Adjunct Lager,2.571435
2,1320,Bud Light,American Light Lager,1.790027
3,41821,Bud Light Lime,American Light Lager,2.078862
4,1341,Keystone Light,American Light Lager,1.716901
5,653,Natural Ice,American Adjunct Lager,1.712514
6,3734,Michelob Ultra,American Light Lager,1.795152
7,65,Budweiser,American Adjunct Lager,2.261485
8,2280,Miller Genuine Draft,American Adjunct Lager,2.183389
9,436,Amstel Light,American Light Lager,2.399250


In [20]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  2512
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Chimay Grande Réserve (Blue), Belgian Strong Dark Ale, score = 4.2537427801309216 are:


,beer_id,name,style,score_mean
0,672,Chimay Première (Red),Belgian Dubbel,4.123422
1,1346,Chimay Tripel (White),Belgian Tripel,4.109288
2,1421,Delirium Nocturnum,Belgian Strong Dark Ale,3.983673
3,1696,Trappistes Rochefort 8,Belgian Strong Dark Ale,4.293923
4,645,Trappistes Rochefort 10,Belgian Quadrupel (Quad),4.452208
5,1385,Delirium Tremens,Belgian Strong Pale Ale,4.089077
6,1946,Franziskaner Hefe-Weisse,German Hefeweizen,3.996700
7,178857,ReDANKulous,American Imperial IPA,4.121856
8,156,Piraat,Belgian Strong Pale Ale,4.027381
9,30,Trois Pistoles,Belgian Strong Dark Ale,4.195384


In [21]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  1352
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Shiner Bock, German Bock, score = 3.0876065065840432 are:


,beer_id,name,style,score_mean
0,1312,Molson Canadian Lager,American Adjunct Lager,2.755008
1,837,Coors Light,American Light Lager,1.928670
2,1351,Yuengling Traditional Lager,American Amber / Red Lager,3.303532
3,171,Dos Equis Amber Lager,Vienna Lager,3.070510
4,1276,Coors Banquet,American Adjunct Lager,2.518194
5,848,Tecate,American Adjunct Lager,2.500783
6,810,Mickey's,American Malt Liquor,2.413893
7,1053,Moosehead Lager,American Lager,2.918225
8,6,Turbodog,English Brown Ale,3.637484
9,2011,Hofbräu Oktoberfestbier,German Märzen / Oktoberfest,3.464604


In [22]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  607
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Fat Tire Belgian Style Ale, American Amber / Red Ale, score = 3.524434306569337 are:


,beer_id,name,style,score_mean
0,32631,Brooklyn Summer Ale,English Pale Ale,3.538762
1,127723,Myron's Walk: Belgian-Style Pale Ale (Beer Cam...,Belgian Pale Ale,3.623322
2,76571,Samuel Adams Alpine Spring,German Kellerbier / Zwickelbier,3.483124
3,74123,Snow Day Winter Ale,American Black Ale,3.668375
4,2249,Hefeweizen,German Hefeweizen,3.492310
5,1426,Grolsch Premium Lager,European Pale Lager,3.052222
6,118062,Saison,Belgian Saison,3.583322
7,127727,Alt Route: Altbier (Beer Camp Across America),German Altbier,3.554654
8,144200,Nooner Pilsner,German Pilsner,3.788012
9,18305,DreamWeaver Wheat,German Hefeweizen,3.713069


In [23]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id =  246
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Heineken Lager Beer, European Pale Lager, score = 2.6418710089399737 are:


,beer_id,name,style,score_mean
0,567,Rolling Rock Extra Pale,American Adjunct Lager,2.500788
1,436,Amstel Light,American Light Lager,2.399250
2,171,Dos Equis Amber Lager,Vienna Lager,3.070510
3,2435,Beck's,German Pilsner,2.713063
4,232,Corona Extra,American Adjunct Lager,2.247109
5,1212,Blue Moon Belgian White,Belgian Witbier,3.261293
6,2755,Cerveza Pacifico Clara,American Adjunct Lager,2.647262
7,449,Stella Artois,European Pale Lager,3.004318
8,667,Negra Modelo,Munich Dunkel Lager,3.285908
9,10071,Sapporo Premium Beer,American Lager,2.791103


In [24]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id = 2755 
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Cerveza Pacifico Clara, American Adjunct Lager, score = 2.647262210796916 are:


,beer_id,name,style,score_mean
0,1907,Dos Equis Special Lager,American Adjunct Lager,2.682793
1,232,Corona Extra,American Adjunct Lager,2.247109
2,848,Tecate,American Adjunct Lager,2.500783
3,20732,Budweiser Select,American Light Lager,2.007452
4,2435,Beck's,German Pilsner,2.713063
5,867,Birra Moretti,European Pale Lager,2.875336
6,34804,Landshark Lager,American Adjunct Lager,2.507511
7,246,Heineken Lager Beer,European Pale Lager,2.641871
8,1320,Bud Light,American Light Lager,1.790027
9,2803,Sol,American Adjunct Lager,2.391426


In [25]:
# Lets test some beers.  Enter a beer and use the predictions model to return 10 nearest neighbors
# K=10, pearson

beer_raw_id = 131
print(f'The 10 nearest neighbors of {get_beer_name(beer_raw_id)}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Celebrator, German Doppelbock, score = 4.2999630390143775 are:


,beer_id,name,style,score_mean
0,144200,Nooner Pilsner,German Pilsner,3.788012
1,945,Adam,English Old Ale,4.259646
2,695,Duvel Belgian Golden Ale,Belgian Strong Pale Ale,4.220689
3,673,Rodenbach Grand Cru,Flanders Red Ale,4.272256
4,225,Burning River Pale Ale,American Pale Ale (APA),3.938050
5,696,California Kölsch,German Kölsch,3.533625
6,224,Schneider Weisse Tap 6 Unser Aventinus,German Weizenbock,4.293003
7,37113,Old Stock Cellar Reserve (Aged In Bourbon Barr...,English Old Ale,4.384061
8,571,Oatmeal Stout,English Oatmeal Stout,4.176924
9,30845,CynicAle,Belgian Saison,3.971047
